# Применение объектно-ориентированных языков программирования для решения задач оптимального управления в модели Солоу


Задача оптимального распределения ограниченных ресурсов при максимизации их полезности от потребления является одной из важнейших задач экономики. В современной экономической ситуации стало возможно найти решение для задачи средствами симулиро-вания имитационных моделей с помощью прикладных программных средств. 
   
   
Пусть производственная функция имеет вид $Y = \alpha_{0} K^{\alpha_{1}} L^{1-\alpha_1}$,то есть $y = f(k) = \alpha_{0} k^{\alpha_1}$ Тогда задача оптимального управления примет вид:*


$\begin{cases}
max \int^{\infty}_{0} e^{-\delta t}u(c(t))dt\\ \\
\frac{dk}{dt} = - \lambda k + \alpha_0 k^{\alpha_1} - c(t), k(0) = k_0 (0)\\ 
0\le c_1 \le c(t) \le \alpha_0 k^{\alpha_1}\\
\end{cases}$


Применив к задаче (1) принцип максимума Понтрягина, получаем, что стационарные траектории  $c' = const$ и $k' = const$, удовлетворяющие условиям   $\frac{dc}{dt} = 0$ и $\frac{dk}{dt} = 0$ , будут являться решениями системы уравнений:

$\begin{cases}
\alpha_0 \alpha_1 k'^{\alpha_1 -1} = \lambda + \delta \\
c' = \alpha_1 k'^{\alpha_1 -1} - \lambda k'\\
\end{cases}$


А именно $k' = (\frac{\alpha_0 \alpha_1}{\lambda+\sigma})^{\frac{1}{1-\alpha_1}}$ и $c' = (\frac{\alpha_0 \alpha_1}{\lambda+\sigma})^{\frac{1}{1-\alpha_1}}(\frac{\lambda+\sigma}{\alpha_1}-\lambda)$    *(2)*


А значения $k_1$ и $k_2$, удовлетворяющие условию $k_1≤k'≤k_2$, найдем как решение уравнения


$f(k) = \alpha_0 k^{\alpha_1} - \lambda k - c_1 = 0$  *(3)*


Рассмотрим для примера модель оптимального управления со следующими исходными данными:


### Импорт библиотек:

In [1]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from plotly import tools
from IPython.display import display, Markdown, Latex
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import numpy as np
%matplotlib nbagg
py.offline.init_notebook_mode(connected = True)


### Вводим начальные параметры для модели

$L_0$ - экономически активное население;


$k_0$ - фондовооруженность труда;


$c_0$ - среднедушевое потребление;


$\alpha_0$ - 


$\alpha_1$ - производственная функция Кобба-Дугласа;


$\mu$ - доля выбытия капитала;


$\delta$ - сила роста, параметр дисконтирования;


$v$ - акселлератор;


$t$ - длина прогноза в годах, брать необходимо 4-х кратные числа из-за високосных годов;


$\Delta t$ - шаг модели, по умолчанию используем день или 1/365.25;


In [2]:
Mari_Data = {'L0':6405, 
             'alpha0':8.7821,
             'alpha1':0.2354,
             'delta':0.1,
             'nu':0.0619,
             'v':0.2373,
             'k0':358656/6405,
             'c0':40800/6405,
             't':8,
             'delta_t':1/365.25}

In [11]:
40800/6405

6.370023419203747

In [3]:
def saveData(DF1, name ='Забыл назвать'):
    fname = str(name)+'.xlsx'
    writer = pd.ExcelWriter(fname, engine='xlsxwriter')
    DF1.to_excel(writer, 'Относительные')
    writer.save()
def getCK(set_dat):
    
    delta =set_dat['delta']
    nu = set_dat['nu']
    alpha0 = set_dat['alpha0']
    alpha1 = set_dat['alpha1']
    r0 = 1- alpha1
    L0 = set_dat['L0']
    v = set_dat['v']
    k0 = set_dat['k0']
    c0 = set_dat['c0']
    max_t = set_dat['t']
    delta_t = set_dat['delta_t']
    K_find = (((alpha0*alpha1)/(nu+v+delta))**(1/r0))
    C_find = K_find * (((nu+v+delta)/alpha1)-(nu+v))
    Y_find = ((alpha0*K_find)**alpha1)
    
    up_bord = K_find
    down_bord = 0
    k1 = 0
    f_k_1 = 0
    while True:
        new_bord = down_bord + (up_bord - down_bord)*0.5
        f_new = (set_dat['alpha0']*(new_bord**alpha1) - (nu+v)*(new_bord) - c0)
        if (f_new>0):
            up_bord = new_bord
        else:
            down_bord = new_bord
        k1 = new_bord
        f_k_1 = (alpha0*(k1**alpha1) - (nu+v)*(k1) - c0)
        if (f_k_1 < 0.00001 and f_k_1 >0):
            break
    up_bord = 100000
    down_bord = K_find
    k2 = K_find
    f_k_2 = 0
    while True:
        new_bord = down_bord + (up_bord - down_bord)*0.5;
        f_new = (alpha0*(new_bord**alpha1) - (nu+v)*(new_bord) - c0)
        if (f_new<0):
            up_bord = new_bord
        else:
            down_bord = new_bord;
        k2 = new_bord
        f_k_2 = (alpha0*(k2**alpha1) - (nu+v)*(k2) - c0)*L0
        if (f_k_2 < 0.00001 and f_k_2 >0):
            break
       
    return C_find, K_find  
def get_roots_K(set_dat):
    delta =set_dat['delta']
    nu = set_dat['nu']
    alpha0 = set_dat['alpha0']
    alpha1 = set_dat['alpha1']
    r0 = 1- alpha1
    L0 = set_dat['L0']
    v = set_dat['v']
    k0 = set_dat['k0']
    c0 = set_dat['c0']
    delta_t = set_dat['delta_t']
    K_find = (((alpha0*alpha1)/(nu+v+delta))**(1/r0))

   
    
    up_bord = K_find
    down_bord = 0
    k1 = 0
    f_k_1 = 0
    while True:
        new_bord = down_bord + (up_bord - down_bord)*0.5
        f_new = (set_dat['alpha0']*(new_bord**alpha1) - (nu+v)*(new_bord) - c0)
        if (f_new>0):
            up_bord = new_bord
        else:
            down_bord = new_bord
        k1 = new_bord
        f_k_1 = (alpha0*(k1**alpha1) - (nu+v)*(k1) - c0)
        if (f_k_1 < 0.00001 and f_k_1 >0):
            break
    up_bord = 100000
    down_bord = K_find
    k2 = K_find
    f_k_2 = 0
    while True:
        new_bord = down_bord + (up_bord - down_bord)*0.5;
        f_new = (alpha0*(new_bord**alpha1) - (nu+v)*(new_bord) - c0)
        if (f_new<0):
            up_bord = new_bord
        else:
            down_bord = new_bord;
        k2 = new_bord
        f_k_2 = (alpha0*(k2**alpha1) - (nu+v)*(k2) - c0)*L0
        if (f_k_2 < 0.00001 and f_k_2 >0):
            break
       
    return k1,k2  
def getData(set_dat, method = 'Потребление'):
    delta = set_dat['delta']
    nu = set_dat['nu']
    alpha0 = set_dat['alpha0']
    alpha1 = set_dat['alpha1']
    r0 = 1- alpha1
    L0 = set_dat['L0']
    v = set_dat['v']
    K0 = set_dat['k0']*L0
    Y0 = (alpha0 * (pow(K0,alpha1))*(L0**r0))
    
    if(method == 'Инвестиции'):
        C0 = set_dat['c0']*L0
    elif(method == 'Потребление'):
        C0 = Y0
    
    I0 = Y0-C0
    max_t = set_dat['t']
    delta_t = set_dat['delta_t']
    length = int(max_t/delta_t) 
    k, y, c, i, time_line = [],[],[],[],[]
    
    for time in range(length+1):
        time_line.append(time*delta_t)
    datelist = pd.date_range(datetime.datetime(2015,1,1), datetime.datetime(2015+max_t,1,1)).tolist() 
    
    L = [L0,]  
    K = [K0,]
    Y = [Y0,]
    C = [C0,]
    I = [I0,]
        
    C_find, K_find = getCK(set_dat)
    
    for n in range(length):
               
        L_exmp = L[n]*math.exp(v*delta_t)
        K_exmp = (1 -nu*delta_t)*K[n] + I[n]*delta_t
        Y_exmp = (alpha0*(K_exmp**alpha1))*math.pow(L_exmp,r0)
        
        
        if(method == 'Инвестиции'):
            if(K_exmp <= K_find*L_exmp):
                C_exmp = C0*L_exmp
            else:
                C_exmp = C_find*L_exmp
                
        elif(method == 'Потребление'):
            if(K_exmp>= K_find*L_exmp):
                C_exmp = Y_exmp
            elif(K_exmp< K_find*L_exmp):
                C_exmp = C_find*L_exmp
            
        
        I_exmp = Y_exmp - C_exmp
        C.append(C_exmp)
        K.append(K_exmp)
        Y.append(Y_exmp)
        I.append(I_exmp)
        L.append(L_exmp)
      
   
    U=[1,]
    for op in range(length):
        mu = ((C[op]/L[op])/((1+delta)**time_line[op]))**delta_t
        sum_u =  mu+U[op]
        U.append(sum_u)
        
    d = {'t' : time_line, 'datelist':datelist,'K' : K, 'Y' : Y, 'C' : C, 'I' : I, 'L' : L, 'U' : U,
         'k':np.array(K)/np.array(L),'i':np.array(I)/np.array(L),
         'c':np.array(C)/np.array(L),'y':np.array(Y)/np.array(L)}
    DF1 = pd.DataFrame(d)           
    return DF1
def makePlt(set_dat,method): 
    
    Test_DF = getData(set_dat, method)
      
    layout = go.Layout(
        title = "Расчет для РМЭ",
        yaxis = dict(title = 'Тыс. руб'),
        xaxis = dict(title = 'Дата'),)
    trace1 = go.Scatter(
        x = Test_DF['datelist'],
        y = Test_DF['k'], 
        mode = 'lines',
        name = 'Фондовооруженность труда',
        line = dict(
        shape = 'spline'))
    trace2 = go.Scatter(
        x = Test_DF['datelist'],
        y = Test_DF['c'], 
        mode = 'lines',
        name = 'Среднедушевое потребление',
        line = dict(
        shape = 'spline'))
    trace3 = go.Scatter(
        x = Test_DF['datelist'],
        y = Test_DF['i'], 
        mode = 'lines',
        name = 'Удельныые инвестиции',
        line = dict(
        shape = 'spline'))
    trace4 = go.Scatter(
        x = Test_DF['datelist'],
        y = Test_DF['y'], 
        mode = 'lines',
        name = 'Производительность труда',
        line = dict(
        shape = 'spline'))    
    fig = go.Figure(data=[trace1,trace2,trace3,trace4], layout = layout)   
    py.offline.iplot(fig)

In [7]:
Mari_Data = {'L0':6405, 'alpha0':8.7821,'alpha1':0.2354,'delta':0.1,
             'nu':0.0619,'v':0.2373,'k0':358656/6405,'c0':61200/6405,
             't':8,'delta_t':1/365.25}
method = 'Потребление'
makePlt(Mari_Data, method)

In [8]:
delta =Mari_Data['delta']
nu = Mari_Data['nu']
alpha0 = Mari_Data['alpha0']
alpha1 = Mari_Data['alpha1']
r0 = 1- alpha1
L0 = Mari_Data['L0']
v = Mari_Data['v']
k0 = Mari_Data['k0']
c0 = Mari_Data['c0']
max_t = Mari_Data['t']
delta_t = Mari_Data['delta_t']
K_find = (((alpha0*alpha1)/(nu+v+delta))**(1/r0))
C_find = K_find * (((nu+v+delta)/alpha1)-(nu+v))
Y_find = ((alpha0*K_find)**alpha1)
k1,k2 = get_roots_K(Mari_Data)
display(Markdown('$Y = {0}\cdot K^{{{1}}}\cdot L^{{{2}}}$'.format(str(alpha0), alpha1, r0)))
display(Markdown('$L_0 = %f$'%L0))
display(Markdown('$K_0 = %f$'%(k0*L0)))
display(Markdown('$C_1 = %f$'%(c0*L0)))
display(Markdown('$\mu = %f$'%nu))
display(Markdown('$v = %f$'%v))
display(Markdown('$\delta = %f$'%((nu+v))))
display(Markdown("""Тогда $c_1 = {{{0}}}, k_0 = {{{1}}}, y = {{{2}}}k^{{{3}}}$ 
                 и уравнение(3) примет вид: """
                 .format(c0,k0,alpha0,alpha1)))
display(Markdown('$y = {{{0}}}k^{{{1}}} - {{{2}}}k - {{{3}}} = 0,$'
                 .format(alpha0,alpha1,(nu+v), c0)))
display(Markdown('корни которого равны $k_1 = {{{0}}}, k_2 = {{{1}}}$'.format(k1,k2)))

$Y = 8.7821\cdot K^{0.2354}\cdot L^{0.7646}$

$L_0 = 6405.000000$

$K_0 = 358656.000000$

$C_1 = 61200.000000$

$\mu = 0.061900$

$v = 0.237300$

$\delta = 0.299200$

Тогда $c_1 = {9.55503512880562}, k_0 = {55.99625292740047}, y = {8.7821}k^{0.2354}$ 
                 и уравнение(3) примет вид: 

$y = {8.7821}k^{0.2354} - {0.2992}k - {9.55503512880562} = 0,$

корни которого равны $k_1 = {1.80814586105752}, k_2 = {36.53399102072706}$

In [9]:
getData(Mari_Data, method)

,t,datelist,K,Y,C,I,L,U,k,i,c,y
0,0.000000,2015-01-01,358656.000000,145088.881536,145088.881536,0.000000,6405.000000,1.000000,55.996253,0.000000,22.652441,22.652441
1,0.002738,2015-01-02,358595.217505,145155.181577,145155.181577,0.000000,6409.162629,2.008579,55.950401,0.000000,22.648073,22.648073
2,0.005476,2015-01-03,358534.445310,145221.511914,145221.511914,0.000000,6413.327964,3.017158,55.904586,0.000000,22.643706,22.643706
3,0.008214,2015-01-04,358473.683415,145287.872562,145287.872562,0.000000,6417.496005,4.025735,55.858809,0.000000,22.639340,22.639340
4,0.010951,2015-01-05,358412.931818,145354.263535,145354.263535,0.000000,6421.666756,5.034310,55.813069,0.000000,22.634975,22.634975
5,0.013689,2015-01-06,358352.190516,145420.684845,145420.684845,0.000000,6425.840217,6.042885,55.767367,0.000000,22.630610,22.630610
6,0.016427,2015-01-07,358291.459508,145487.136507,145487.136507,0.000000,6430.016390,7.051458,55.721702,0.000000,22.626247,22.626247
7,0.019165,2015-01-08,358230.738793,145553.618535,145553.618535,0.000000,6434.195278,8.060030,55.676075,0.000000,22.621884,22.621884
8,0.021903,2015-01-09,358170.028368,145620.130943,145620.130943,0.000000,6438.376881,9.068600,55.630485,0.000000,22.617522,22.617522
9,0.024641,2015-01-10,358109.328232,145686.673745,145686.673745,0.000000,6442.561202,10.077170,55.584932,0.000000,22.613161,22.613161
